<a href="https://colab.research.google.com/github/rubaahmedkhan/Gemini-Experiments/blob/main/langchain1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Google Gemini API with LangChain Project**


## **Install Required Package**
**LangChain:** A framework for building applications using large language models, facilitating the creation of language model pipelines.

**LangChain Google GenAI Integration**: An integration with Google's Generative AI tools, allowing for the use of advanced language models within the LangChain framework.

In [14]:
# Install the LangChain and LangChain's Google GenAI integration
# `-q` keeps the output minimal.
# `-U` ensures you are using the latest versions of the package

!pip install -q -U google-generativeai
!pip install -qU langchain-google-genai

In [2]:
import langchain_google_genai as genai

In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI


In [3]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')

In [11]:
mode: ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    api_key=GOOGLE_API_KEY)

# **Invocation**

In [22]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to urdu. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = mode.invoke(messages)
ai_msg

AIMessage(content='مجھے پروگرامنگ سے بہت پیار ہے۔ (mujhay programming say bohat pyar hai)', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-3cb33a5d-20cf-4234-8d1f-dbb4882d8410-0', usage_metadata={'input_tokens': 20, 'output_tokens': 25, 'total_tokens': 45, 'input_token_details': {'cache_read': 0}})

In [23]:
print(ai_msg.content)

مجھے پروگرامنگ سے بہت پیار ہے۔ (mujhay programming say bohat pyar hai)


# **Chaining**
We can chain our model with a prompt template like so:



from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke(
    {
        "input_language": "English",
        "output_language": "German",
        "input": "I love programming.",
    }
)

In [28]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)
ai_msg = mode.invoke(messages)
ai_msg

chain = prompt | mode
chain.invoke(
    {
        "input_language": "urdu",
        "output_language": "english",
        "input": "mujhy nehari pasamd hai",
    }
)

AIMessage(content='I like Nihari.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-44d0a013-e63e-4bbc-95e1-aec478d451cf-0', usage_metadata={'input_tokens': 19, 'output_tokens': 6, 'total_tokens': 25, 'input_token_details': {'cache_read': 0}})

# **Image input:**

In [29]:
import base64
import httpx
from langchain_core.messages import HumanMessage

image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
image_data = base64.b64encode(httpx.get(image_url).content).decode("utf-8")
message = HumanMessage(
    content=[
        {"type": "text", "text": "describe the weather in this image"},
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
        },
    ]
)
ai_msg = mode.invoke([message])
ai_msg.content

"The weather in the image appears to be sunny and clear.  The sky is mostly blue with a few scattered, wispy clouds.  The sunlight seems bright enough to cast shadows, although there aren't any strong shadows visible in the image, suggesting the sun might be relatively high in the sky. The overall impression is of a pleasant, warm day."

In [36]:
messages = [
    ("system", "Translate the user sentence to urdu."),
    ("human", "I love programming."),
]
mode.invoke(messages)

AIMessage(content='مجھے پروگرامنگ بہت پسند ہے۔ (mujhe programming bohat pasand hai)', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-8732fa5f-0bdf-4eea-b2c6-44c94f2e1c8c-0', usage_metadata={'input_tokens': 11, 'output_tokens': 23, 'total_tokens': 34, 'input_token_details': {'cache_read': 0}})

# **Stream:**

In [42]:
for chunk in mode.stream(messages):
    #print(chunk)
    print(chunk.content)

مج
ھے پروگرامنگ بہت پسند ہے۔ (mujhe programming
 bohat pasand hai)


In [33]:
stream = mode.stream(messages)
full = next(stream)
for chunk in stream:
    full += chunk
full

AIMessageChunk(content="J'aime la programmation.", additional_kwargs={}, response_metadata={'safety_ratings': [], 'finish_reason': 'STOP'}, id='run-29ee4187-eda7-4a2e-aa6b-8fe9bab79804', usage_metadata={'input_tokens': 11, 'output_tokens': 7, 'total_tokens': 18, 'input_token_details': {'cache_read': 0}})